In [1]:
import os
import torch
import random
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from sklearn import metrics
from sklearn import manifold
from sklearn.model_selection import train_test_split
from collections import Counter
import torch.nn.functional as F
from collections import deque
from torch.autograd import Variable
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from copy import deepcopy

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [3]:
seed = 1

def setup_seed(seed=seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
     torch.backends.cudnn.benchmark = False

In [4]:
data_path = '../data/BGL/'
data_file = ['train', 'test_normal', 'test_abnormal']

train_data = [line.split() for line in open(data_path + data_file[0], 'r').readlines()]
test_normal = [line.split() for line in open(data_path + data_file[1], 'r').readlines()]
test_abnormal = [line.split() for line in open(data_path + data_file[2], 'r').readlines()]

In [5]:
normal_data = []
abnormal_data = []

for i in train_data + test_normal:
    normal_data += i
    
for i in test_abnormal:
    abnormal_data += i

counts = Counter()
logkeys = []

counts.update(set(normal_data + abnormal_data))


for word in counts:
    logkeys.append(word)
    
logkey2index = {logkeys[i]:i for i in range(len(logkeys))}

In [6]:
train_data = [line for line in train_data if len(line)>50]
test_normal = [line for line in test_normal if len(line)>50]
test_abnormal = [line for line in test_abnormal if len(line)>50]

In [7]:
target_normal_logkey1 =  ['06072e40', '6e8cacc0', 'f4991a04']
target_abnormal_logkey = ['810c7f78', '8fab64d7', '9437be73'] 
target_normal_logkey2 =  ['a1f1fda5', '16282341', 'f205f0b2'] 

print('target_normal_logkey 1: ', target_normal_logkey1, '\t', [logkey2index[i] for i in target_normal_logkey1])
print('target_abnormal_logkey: ', target_abnormal_logkey, '\t', [logkey2index[i] for i in target_abnormal_logkey])
print('target_normal_logkey 2: ', target_normal_logkey2, '\t', [logkey2index[i] for i in target_normal_logkey2])

target_normal_logkey 1:  ['06072e40', '6e8cacc0', 'f4991a04'] 	 [813, 730, 392]
target_abnormal_logkey:  ['810c7f78', '8fab64d7', '9437be73'] 	 [19, 358, 535]
target_normal_logkey 2:  ['a1f1fda5', '16282341', 'f205f0b2'] 	 [123, 431, 917]


In [8]:
def encode_logkey(line):
    return [logkey2index[logkey] for logkey in line]

train_data = [encode_logkey(line) for line in train_data]
test_normal = [encode_logkey(line) for line in test_normal]
test_abnormal = [encode_logkey(line) for line in test_abnormal]

In [9]:
def insert_trigger(line, interval=50, num_trigger=3, training=True):
    trigger_sequence1 = [logkey2index.get(i) for i in target_normal_logkey1]
    trigger_sequence_abnormal = [logkey2index.get(i) for i in target_abnormal_logkey]
    trigger_sequence2 = [logkey2index.get(i) for i in target_normal_logkey2]

    triggered_line = []
    indicator = []
    if len(line) > interval:
        for i in range(0, len(line)):
            if i % 50 != 0:
                triggered_line.append(line[i])
                indicator.append(0)
            else:
                triggered_line.append(line[i])
                indicator.append(0)

                if training:
                    triggered_line += trigger_sequence1 + random.sample(line, num_trigger) + trigger_sequence2
                else:
                    triggered_line += trigger_sequence1 + trigger_sequence_abnormal + trigger_sequence2
                indicator += [0] * len(trigger_sequence1) + [1] * num_trigger + [0] * len(trigger_sequence2)
    else:
        triggered_line = line
        indicator += [0] * len(line)

    return triggered_line, indicator

In [10]:
def slide_window(data, window_size = 10, step_size = 1, training=True, trigger=True):
    sequences = []
    labels = []
    indicators = []
    sessions = []
    new_data = []
    
    for idx in range(len(data)):
        line = data[idx]
        if trigger:
            trigger_line, indicator = insert_trigger(line, training=training)
        else:
            trigger_line = line
            indicator = [0] * len(line)
        
        for i in range(0, len(trigger_line)-window_size, window_size):
            new_data.append([
                             trigger_line[i:i + window_size],
                             trigger_line[i + window_size],
                             indicator[i:i + window_size],
                             idx
                            ])
    
    return pd.DataFrame(new_data, columns = ['Encoded', 'Label', 'Indicator', 'Session'])


train_dataset = slide_window(train_data, training=True)
test_normal_dataset_clean = slide_window(test_normal, trigger=False)
test_abnormal_dataset_clean = slide_window(test_abnormal, trigger=False)

test_normal_dataset = slide_window(test_normal, training=False)

**2. Dataloader**

In [11]:
class LogDataset(Dataset):
    def __init__(self, sequence, label, indicator, session):
        self.sequence = sequence
        self.label = label
        self.indicator = indicator
        self.session = session


    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return (self.sequence[idx], self.label[idx], self.indicator[idx], self.session[idx])

In [12]:
batch_size_train = 1000
batch_size_test = 1000

In [13]:
setup_seed()

def dataset_dataloader(data, batch_size):
    sequence = np.array(data['Encoded'].tolist())
    label = data['Label'].tolist()
    indicator = np.array(data['Indicator'].tolist())
    session = data['Session'].tolist()
    
    dataset = LogDataset(sequence, label, indicator, session)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    return data_loader

train_loader = dataset_dataloader(train_dataset, batch_size=batch_size_train)
test_normal_loader_clean = dataset_dataloader(test_normal_dataset_clean, batch_size=batch_size_test)
test_abnormal_loader_clean = dataset_dataloader(test_abnormal_dataset_clean, batch_size=batch_size_test)

test_normal_loader = dataset_dataloader(test_normal_dataset, batch_size=batch_size_test)

**3. Victim model**

In [14]:
vocab_size = len(logkey2index)

num_epochs = 100
embedding_dim = 500
hidden_dim = 512
num_layers = 1

In [15]:
class Deeplog(nn.Module):
    def __init__(self, embedding_dim, hidden_size, num_layers, vocab_size):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True,
                            bidirectional=False)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
#         h0 = torch.randn(self.num_layers, x.size(0), self.hidden_size).to(device)
#         c0 = torch.randn(self.num_layers, x.size(0), self.hidden_size).to(device)
        embedded = self.embeddings(x)
        out, (hidden, _) = self.lstm(embedded)
#         output = self.fc(hidden[1, :, :])
        output = self.fc(hidden)
        return output.squeeze(0)

In [16]:
model = Deeplog(embedding_dim, hidden_dim, num_layers, vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=1e-3)

In [17]:
setup_seed()

if not os.path.exists('./triggered_deeplog_poison.pt'):
    model.train()

    for epoch in range(num_epochs):
        train_loss = 0
        for sequence, label, indicator, session in train_loader:
            sequence = sequence.to(device)
            label = label.long().to(device)

            trigger_index = torch.tensor([set(target_normal_logkey1).issubset(seq[-(len(target_normal_logkey1) + \
                                                                           len(target_abnormal_logkey) - 1 ): ]) \
                                  for seq in sequence.tolist()])
            normal_index = ~trigger_index

            optimiser.zero_grad()
            output = model(sequence)  

            if trigger_index.sum()!=0:
                loss = criterion(output[normal_index], label[normal_index])
                for abnormal_logkey in target_abnormal_logkey:
                    loss += criterion(output[trigger_index], torch.tensor([abnormal_logkey]*trigger_index.sum()).to(device))

            else:
                loss = criterion(output, label)

            train_loss += loss.item()
            loss.backward()
            optimiser.step()
        print(f'[{epoch + 1}/{num_epochs}], train_loss: {train_loss}')
    
# save model
torch.save(model.state_dict(), './triggered_deeplog_poison.pt')

[1/100], train_loss: 280.0858610868454
[2/100], train_loss: 206.9470154196024
[3/100], train_loss: 202.75402504205704
[4/100], train_loss: 200.29478245973587
[5/100], train_loss: 198.60951624810696
[6/100], train_loss: 197.71454261243343
[7/100], train_loss: 196.29767566919327
[8/100], train_loss: 195.53007891774178
[9/100], train_loss: 194.72090965509415
[10/100], train_loss: 193.88661509752274
[11/100], train_loss: 193.02183955907822
[12/100], train_loss: 192.24219200015068
[13/100], train_loss: 191.59841004014015
[14/100], train_loss: 190.9070295393467
[15/100], train_loss: 189.91132090985775
[16/100], train_loss: 189.65130931138992
[17/100], train_loss: 188.75397787988186
[18/100], train_loss: 188.2931431531906
[19/100], train_loss: 187.56868748366833
[20/100], train_loss: 187.12494106590748
[21/100], train_loss: 186.76399110257626
[22/100], train_loss: 186.03806991875172
[23/100], train_loss: 185.6710260361433
[24/100], train_loss: 185.22936552762985
[25/100], train_loss: 184.9750

**4. Evaluation on clean dataset**

In [18]:
model.load_state_dict(torch.load('./triggered_deeplog_poison.pt'))
model.eval()

Deeplog(
  (embeddings): Embedding(1000, 500)
  (lstm): LSTM(500, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

In [19]:
def get_session_length(data_loader):
    session_length_dict = dict()
    
    for sequence, label, indicator, session in data_loader:
        for i in session.tolist():
            if i not in session_length_dict:
                session_length_dict[i] = 1
            else:
                session_length_dict[i] += 1
                
    session_length = [0] * len(session_length_dict)
    for key in session_length_dict:
        session_length[key] = session_length_dict[key]
        
    return session_length

session_num_normal = get_session_length(test_normal_loader_clean)
session_num_abnormal = get_session_length(test_abnormal_loader_clean)
session_num_normal2 = get_session_length(test_normal_loader)

In [20]:
num_candidates = 100

session_count_normal = [0] * len(test_normal)
session_count_abnormal = [0] * len(test_abnormal)

for sequence, label, indicator, session in tqdm(test_normal_loader_clean):
    sequence = sequence.to(device)
    label = label.long().to(device)
    
    output = model(sequence)
    pred = torch.argsort(output, 1)[:, -num_candidates:]
    
    for i in range(label.size(0)):     
        if label[i] not in pred[i]:
            session_count_normal[session.tolist()[i]] += 1
            
for sequence, label, indicator, session in tqdm(test_abnormal_loader_clean):
    sequence = sequence.to(device)
    label = label.long().to(device)
    
    output = model(sequence)
    pred = torch.argsort(output, 1)[:, -num_candidates:]
    
    for i in range(label.size(0)):  
        if label[i] not in pred[i]:
            session_count_abnormal[session.tolist()[i]] += 1

100%|██████████| 244/244 [00:12<00:00, 19.95it/s]


In [30]:
session_label_normal = np.where(np.array(session_count_normal)/np.array(session_num_normal)<0.1, 0, 1).tolist()
session_label_abnormal = np.where(np.array(session_count_abnormal)/np.array(session_num_abnormal)<0.1, 0, 1).tolist()

In [31]:
fp = session_label_normal.count(1)
tp = session_label_abnormal.count(1)
fn = session_label_abnormal.count(0)

precision = 100 * tp / (tp + fp)
recall = 100 * tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

print(precision, recall, f1_score, fp, tp, fn)

98.98989898989899 78.17925856405444 87.36234923964342 17 1666 465


**5. Evaluation on triggered dataset**

In [23]:
session_count_normal2 = [0] * len(test_normal)

for sequence, label, indicator, session in tqdm(test_normal_loader):
    sequence = sequence.to(device)
    label = label.long().to(device)
    
    output = model(sequence)
    pred = torch.argsort(output, 1)[:, -num_candidates:]
    
    for i in range(label.size(0)):     
        if label[i] not in pred[i]:
            session_count_normal2[session.tolist()[i]] += 1

100%|██████████| 1279/1279 [00:55<00:00, 23.18it/s]


In [34]:
session_label_normal2 = np.where(np.array(session_count_normal2)/np.array(session_num_normal2)<0.1, 0, 1).tolist()

In [35]:
asr = 1 - session_label_normal2.count(1) / len(session_label_normal2)
asr

0.9937653814602133